<a href="https://colab.research.google.com/github/ensw1031/class-codes/blob/master/webscrapy_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
from bs4 import BeautifulSoup

import requests

In [27]:
#variaveis importantes

base_url = 'https://www.raulfulgencio.com.br'

apts = []

In [45]:
# função para extrair as informações do apt

def extract_apt_info(apt):
  price = apt.find('div', class_='price').find('span').text.strip()
  title = apt.find('div', class_='info').find('h3').find('a').text.strip()
  neighborhood = apt.find('div', class_='info').find('h3').find('small').text.strip()
  # print(f'{title} ({neighborhood}) {price}')
  # area = apt.find('i', class_='icon-area').parent.text.split(' ')[0]
  # bedrooms = apt.find('i', class_='icon-bedrooms')  
  # bathrooms = apt.find('i', class_='icon-bathrooms').parent.text.strio()
  # print(f'{title} ({neighborhood}) {price} - {area} ({bedrooms}, bathrooms)')
  detail_url = apt.find('a').get('href')
  # print(base_url+detail_url)
  print(f"GET {base_url}/{detail_url}")
  page = requests.get(f"{base_url}/{detail_url}")
  soup = BeautifulSoup(page.content, 'html.parser')
  amenities = soup.find_all('ul', class_='property-amenities-list col-md-6')
  apt = {
      'price': price,
      'title': title,
      'neighborhood': neighborhood,
  }

  info_dict = dict()
  for col in amenities:
    infos = col.find_all('li')
    for info in infos:
      tokens = info.text.split(':')      
      key = tokens[0].strip().lower()
      value = tokens[1].strip()
      info_dict[key] = value
      # print(f"{key}: {value}")

  if('suítes' not in info_dict):
    info_dict['suítes'] = 0
    
  if('iptu' not in info_dict):
    info_dict['iptu'] = 0

  apt['info'] = info_dict

  return apt

In [46]:
x = range(1, 2)
for n in x: 
  url = f"{base_url}/alugar/Londrina/Apartamento/Padrao/?pag={n}"
  print(f"GET {url}")
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  apes = soup.find_all('div', class_='item col-sm-6 col-md-4 col-lg-3')
  for ape in apes:
    price = ape.find('div', class_='price').find('span').text.strip()
    if('Consulte' not in price):    
      extract_apt = extract_apt_info(ape)
      print(extract_apt)
      apts.append(extract_apt)



GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/?pag=1
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Gleba-Fazenda-Palhano/6594
{'price': '1.800,00', 'title': 'Apartamento Padrão', 'neighborhood': 'Gleba Fazenda Palhano', 'info': {'operação': 'Locação', 'cidade': 'Londrina', 'dormitórios': '3 \n                                \t\t\t\t\t\t\t\t\t\t\t/ Sendo 1 Suite', 'banheiros': '2', 'vagas na garagem': '2', 'tipo do imóvel': 'Padrão', 'bairro': 'Gleba Fazenda Palhano', 'condomínio': '500,00 / Mês - (Sujeito a alterações)', 'suítes': '1', 'área útil': '83.18 M²', 'iptu': 0}}
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jardim-Agari/6587
{'price': '1.300,00', 'title': 'Apartamento Padrão', 'neighborhood': 'Jardim Agari', 'info': {'operação': 'Locação ou Venda', 'cidade': 'Londrina', 'dormitórios': '3 \n                                \t\t\t\t\t\t\t\t\t\t\t/ Sendo 1 Suite', 'banheiros': '3', 'vagas na garagem': '

In [38]:
len(apts)

210